## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-10-22-34-38 +0000,bbc,Trump demands homeless people 'immediately' mo...,https://www.bbc.com/news/articles/c1kzz4m3ll1o...
1,2025-08-10-22-29-46 +0000,bbc,Four Al Jazeera journalists killed in Israeli ...,https://www.bbc.com/news/articles/ceqyyrp3yq9o...
2,2025-08-10-22-23-40 +0000,nypost,Bumbling adult trapped inside Chuck E. Cheese ...,https://nypost.com/2025/08/10/us-news/californ...
3,2025-08-10-22-00-00 +0000,wsj,Activist Investor to Push Avantor to Sell Itself,https://www.wsj.com/business/deals/activist-in...
4,2025-08-10-21-51-50 +0000,nypost,‘Rich kids’ from ritzy UES school granted righ...,https://nypost.com/2025/08/10/us-news/birch-wa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2353/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,28
115,putin,15
117,ukraine,13
16,gaza,9
28,california,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
96,2025-08-10-14-15-48 +0000,nyt,"In a Trump-Putin Summit, Ukraine Fears Losing ...",https://www.nytimes.com/2025/08/10/us/politics...,72
170,2025-08-10-01-00-00 +0000,wsj,President Trump has said Vice President JD Van...,https://www.wsj.com/politics/policy/who-is-the...,69
175,2025-08-09-23-34-20 +0000,nypost,White House weighs inviting Zelensky to Trump’...,https://nypost.com/2025/08/09/us-news/white-ho...,69
80,2025-08-10-16-41-53 +0000,nyt,Dmitri Kozak Was a Key Putin Aide. He Lost Pow...,https://www.nytimes.com/2025/08/10/world/europ...,69
46,2025-08-10-19-52-29 +0000,nypost,Vance says only Trump can decide when to bring...,https://nypost.com/2025/08/10/us-news/vance-sa...,69


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
96,72,2025-08-10-14-15-48 +0000,nyt,"In a Trump-Putin Summit, Ukraine Fears Losing ...",https://www.nytimes.com/2025/08/10/us/politics...
84,45,2025-08-10-16-15-23 +0000,nypost,Netanyahu lays out Gaza City incursion goals a...,https://nypost.com/2025/08/10/world-news/netan...
80,41,2025-08-10-16-41-53 +0000,nyt,Dmitri Kozak Was a Key Putin Aide. He Lost Pow...,https://www.nytimes.com/2025/08/10/world/europ...
24,34,2025-08-10-20-57-00 +0000,wsj,Intel CEO Lip-Bu Tan is set to visit the White...,https://www.wsj.com/tech/intel-ceo-singled-out...
8,26,2025-08-10-21-39-00 +0000,wsj,The Trump administration is interviewing candi...,https://www.wsj.com/politics/policy/trump-admi...
128,26,2025-08-10-10-00-00 +0000,latimes,California's wildfire moonshot: How new techno...,https://www.latimes.com/environment/story/2025...
170,25,2025-08-10-01-00-00 +0000,wsj,President Trump has said Vice President JD Van...,https://www.wsj.com/politics/policy/who-is-the...
12,23,2025-08-10-21-31-20 +0000,nypost,NY Gov. Kathy Hochul slights Zohran Mamdani in...,https://nypost.com/2025/08/10/us-news/ny-gov-k...
1,21,2025-08-10-22-29-46 +0000,bbc,Four Al Jazeera journalists killed in Israeli ...,https://www.bbc.com/news/articles/ceqyyrp3yq9o...
30,21,2025-08-10-20-42-37 +0000,nypost,"US tourist Kevin Mares, 25, shot dead during P...",https://nypost.com/2025/08/10/us-news/us-touri...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
